# Context
Term deposits are a key income source for banks, and telephonic marketing remains one of the most effective ways to sell them. However, these campaigns are resource-intensive, involving large call centers. To optimize efficiency, it's crucial to identify customers who are most likely to convert before reaching out. [This dataset](https://www.kaggle.com/datasets/prakharrathi25/banking-dataset-marketing-targets/data) is related to direct telephonic marketing campaigns of a Portuguese bank, with the goal of predicting whether a customer will subscribe to a term deposit (yes/no). ***My own personal objectives are listed below.***

## Objectives
- Evaluate effectiveness of past campaign... Past Conversion Rate.
- Understand the nature of factors that actually drive conversion
- Build a propensity model to estimate likelihood of future conversion for similar campaign.
- Calibrate model to better reflect probabilities
- Identify top percentage of leads expected to convert, without triggering diminishing returns.
- Measure gains and uplift
- Design and recommend data-informed A/B experiments strategies. 

**NOTE**: This entire analysis is part of my weekly series in efforts to **demystify applied statistical techniques through real-world, project-driven examples**, making concepts like propensity modelling, causal inference, and evaluation metrics more accessible to practitioners of all backgrounds.

Let's connect! --> [LinkedIn](https://www.linkedin.com/in/einstein-ebereonwu/) | [X](https://x.com/einsteinmuna) | [GitHub.](https://github.com/munas-git)

# Importing Important Libraries

In [ ]:
!pip install rich==13.7.1 toolz==0.11.2 scikit-learn==1.4.0 imbalanced-learn==0.12.0 --quiet

In [ ]:
# Data wrangling
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Data vis
import seaborn as sns
import matplotlib.pyplot as plt

# Model training & evaluation
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    precision_score, recall_score, confusion_matrix,
    roc_auc_score, roc_curve, precision_recall_curve,
    classification_report)

# Model explainability
import shap

# Data augmentation
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

# Model calibration
from sklearn.metrics import brier_score_loss
from sklearn.isotonic import IsotonicRegression
from sklearn.calibration import CalibratedClassifierCV, calibration_curve

# Handling warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
train_df = pd.read_csv("/kaggle/input/banking-dataset-marketing-targets/train.csv", sep=";")

# The dataset description says -->
# test.csv: 4521 rows and 18 columns with 10% of the examples (4521)
# randomly selected from train.csv

# For the reason above, I am not working with test.csv at all due to data leakage concerns.

In [ ]:
train_df.head()

# Some EDA

In [ ]:
train_df.info()

In [ ]:
train_df.describe()

**Observation**   
Before even going further, It's worth discussing some extreme values, my assumptions, why and how I am handling some very extreme values.

- ***age:*** Upon further review, the dataset consists of people between 80 - 90+ who also get/don't get the term deposit, and I guess a 95 year could plausibly get a term deposits as some sort of inheritance so I will keep this observation. 
- ***balance:*** A quick gogle search reviewed the following `Yes, Portugal experienced a financial crisis between May 2008 and November 2010` which is the record yer of this dataset so a negatie balance that high is also quite plausible.
- ***duration:*** The dataset metadata describes duration as `last contact duration, in seconds (numeric)`. I don't know about you, but I would never be on a marketing call for *4918* seconds which equates to 81 minutes. I am dropping every observation this extreme.
- ***campaign:*** described as `number of contacts performed during this campaign and for this client (numeric, includes last contact)`. The mean contact is 2 with a reasonable standard deviation, and 75th percentile of 3. 63 contacts (calls) just seems impossible, I will be dropping observations this extreme.
- ***pdays:*** Defined as `number of days that passed by after the client was last contacted from a previous campaign (numeric, -1 means client was not previously contacted)`. I will leave no contact as -1 and temporarily allow last campaign contact that extreme (more than 2 years ago). If I notice significant damage in propensity model, I will also exclude such extreme values.
- ***previous:*** Defined as `number of days that passed by after the client was last contacted from a previous campaign (numeric, -1 means client was not previously contacted)`. When I sorted the dataset in descending order of previous, I noticed only 1 extreme outlier. I am excluding this from the rest of my work.

In [ ]:
# Use this to observe top or bottom values of the data columns to
# further understand my reasons for the actions above.

column_name = 'previous'
numbers_to_return = 3
top_bottom = 2 # specify 1 for ascending order sort and 2 for descending order.. Defaults to Ascending. 

train_df.sort_values(by = column_name, ascending = True if top_bottom == 1 else False if top_bottom == 2 else 1).head(numbers_to_return)

In [ ]:
# Handling the outliers from above

# Drop rows where duration exceeds 99.9th percentile
duration_99p9th = train_df['duration'].quantile(0.999)
print(f'Cut off value is {duration_99p9th}')
print(f'Number of *duration* outliers: {len(train_df[train_df["duration"] >= duration_99p9th])}\n')
train_df = train_df[train_df['duration'] <= duration_99p9th]

# Drop rows where campaign exceeds the 99.9th percentile
campaign_99p9th = train_df['campaign'].quantile(0.999)
print(f'Cut off value is {campaign_99p9th}')
print(f'Number of *campaign* outliers: {len(train_df[train_df["campaign"] >= campaign_99p9th])}\n')
train_df = train_df[train_df['campaign'] <= campaign_99p9th]

# Drop the single max row
previous_max = train_df['previous'].max()
print(f'Cut off value is {previous_max}')
print(f'Number of *previous* outliers: {len(train_df[train_df["previous"] == previous_max])}\n')
train_df = train_df[train_df['previous'] < previous_max]

In [ ]:
train_df.describe()

In [ ]:
train_df.sort_values(by = 'duration', ascending = False).head(3)

**Observation**   
Everything makes much more sense now... The `max duration` is still kind of high at 2087 seconds ~= 35 minutes but I will accept that under the guise that the person on the receiving end was just curious about all the details and wanted to know exactly what they were getting into.

If you notice, the person who had the longest call holds a ***manager*** level job so it makes sense that they are quizing the marketer and want to know as much information before making a strategic financial decision. Their balance is quite low which is suspecious but the person may have other bank accounts while this one is strictly for very small expenses, assuming the amount column isnt expressed in thousands. Same with the blue-collar worked who follows after. The person doesn't seem to have much money so would be wise to know all details before taking the financial decision.

In [ ]:
train_df.info()

# Evaluating effectiveness of past campaign

In [ ]:
past_conversion = train_df['y'].value_counts(normalize=True).values[1]
print(f'The conversion rate for the past campaign was {past_conversion * 100:.3}%')

**Note**   
10% - 12% is a reasonable conversion rate for random targeting... Let's investigate what this looks like across the different covariates.

In [ ]:
categorical_columns = train_df.select_dtypes(include = ['object']).columns.drop('y')

sns.set(style = "whitegrid")
plt.figure(figsize = (12, 12))

for i, col in enumerate(categorical_columns, 1):
    plt.subplot(3, 3, i)
    sns.countplot(x = col, hue = 'y', data = train_df)
    plt.title(f'{col.title()} vs Subscription to Term Deposit')
    plt.xlabel(col)
    plt.ylabel('Count')
    plt.xticks(rotation = 45, ha = 'right')
    plt.legend(title = 'Subscription', labels = ['No', 'Yes'])

plt.tight_layout()
plt.show()

**Observations**   

For the categorical columns above:
- ***Job***: Those assuming managerial positions converted the most as a result of the past campaign, while those who are entrepreneurs or housemaids converted the least.

- ***Marital***: Single individuals had the highest term deposit subscription rate, while divorced individuals had the lowest.

- ***Education***: Individuals with tertiary education had the highest subscription rate, while those with primary education had the lowest.

- ***Default***: Clients with no credit default had significantly higher subscriptions; those with defaults rarely subscribed.

- ***Housing***: Clients without housing loans subscribed more compared to those with housing loans.

- ***Loan***: Clients without personal loans had the highest subscription rate, whereas those with personal loans subscribed much less.

- ***Contact***: Clients contacted via cellular phones subscribed the most, while those contacted via telephone had the lowest turnout.

- ***Month***: The month of May saw the highest number of subscriptions, while March and December had the lowest.

- ***Poutcome***: Clients with a successful previous campaign outcome had the highest subscription rate; those with an outcome marked "failure" or "other" had lower turnout, and those with "unknown" had the least.

In [ ]:
int_columns = train_df.select_dtypes(include = ['int64']).columns

n_cols = 3
n_rows = int(np.ceil(len(int_columns) / n_cols))

plt.figure(figsize=(n_cols * 5, n_rows * 4))

for i, col in enumerate(int_columns, 1):
    plt.subplot(n_rows, n_cols, i)
    sns.histplot(data = train_df, x = col, hue = 'y', bins = 30, kde = True,
                 palette = 'Set2', element = 'step', stat = 'density', common_norm = False)
    plt.title(f'{col.title()} by Subscription to Term Deposit')
    plt.xlabel(col)
    plt.ylabel('Density')
    plt.tight_layout()

plt.suptitle('Integer Feature Distributions by Subscription Outcome', fontsize = 16, y = 1.02)
plt.show()

**Observations**   

For the numerical columns above:
- ***Age***: Subscribers were generally older than non-subscribers, with a noticeable density in the 30–60 age range for successful subscriptions.

- ***Balance***: Clients with higher account balances showed a higher likelihood of subscribing; non-subscribers were concentrated around lower balances.

- ***Day***: There is no strong trend across days of the month; both subscribers and non-subscribers are relatively evenly distributed.

- ***Duration***: Call duration is highly influential, longer call durations were strongly associated with successful subscriptions.

- ***Campaign***: Fewer campaign contacts (1–3 times) were more effective; the probability of subscription decreased as the number of contacts increased.... That is sensible because if they didn't subscribe during 3 contacts then it's probably because they are really not interested.

- ***Pdays***: A high subscription rate is observed when clients had been contacted recently (low pdays values), howeverrr.. some people still convert even after pro-longed periods of no contact, this suggests strong interest/motivation independent of contact recency, or some other factors maybe improved product knowledge, financial goals, friends influence? who knows..


- ***Previous***: Those who had been contacted fewer times in past campaigns (especially 1–2 times) had higher success rates. The probability drops with increasing previous contacts.


# Feature Engineering

In [ ]:
# Encoding all categorical variables.
# Excluding education and month because education level is ordinal data.
# Same as month. Instead of one-hot or categorical, I will 
# use 1 - 12 because the kde for months/target shows seasonal trend.

def preprocess_data(train_df, target_col = 'y'):
    non_ordinal_categorical_columns = categorical_columns.drop(['education', 'month'])
    
    # Dictionary to store fitted label encoders
    label_encoders = {}
    
    # Process non-ordinal categorical columns
    for col in non_ordinal_categorical_columns:
        le = LabelEncoder()
        train_df[col] = le.fit_transform(train_df[col])
        label_encoders[col] = le

    # Ordinal mappings
    month_order = {
        'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'may': 5, 'jun': 6,
        'jul': 7, 'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12}
    
    education_mapping = {
        'primary': 0, 'secondary': 1,
        'tertiary': 2, 'unknown': -1}

    # Apply ordinal mappings
    train_df['month'] = train_df['month'].map(month_order)
    train_df['education'] = train_df['education'].map(education_mapping)

    # Encode target variable
    train_df[target_col] = train_df[target_col].apply(lambda x: 1 if x == 'yes' else 0)
    
    return train_df, label_encoders


train_df, label_encoders = preprocess_data(train_df)

In [ ]:
# All categorical info have now been encoded and ready for propensity modelling
train_df.head()

# Understand the nature of factors that actually drive conversion

In [ ]:
X = train_df.drop(columns='y')
y = train_df['y']


X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size = 0.2,
    random_state = 42,
    stratify = y)

X_train, X_calib, y_train, y_calib = train_test_split(
    X_train, y_train,
    test_size = 0.2,
    random_state = 42,
    stratify = y_train)

print(f'Train Set split: {y_train.value_counts()}')
print(f'\nCalibration Set split: {y_calib.value_counts()}')
print(f'\nTest Set split: {y_test.value_counts()}')

## Training model for initial feature relevance & performance observation

In [ ]:
# This is just to get a rough understanding of what factors influence models understanding
# Of the data.... I will train a better optimised model based on the results obtained here

rf_model = RandomForestClassifier(
    n_estimators = 300,
    max_depth = 10,
    class_weight = 'balanced',
    random_state = 42
)
rf_model.fit(X_train, y_train)

y_pred = rf_model.predict(X_test)
y_proba = rf_model.predict_proba(X_test)[:, 1]

conf_matrix = confusion_matrix(y_test, y_pred)
fpr, tpr, _ = roc_curve(y_test, y_proba)
roc_auc = roc_auc_score(y_test, y_proba)
precision, recall, _ = precision_recall_curve(y_test, y_proba)

print(f'\nClassification Report:\n {classification_report(y_test, y_pred)}\n\n')

plt.figure(figsize = (18, 5))
plt.subplot(1, 3, 1)
sns.heatmap(conf_matrix, annot = True, fmt = 'd', cmap = 'Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')

plt.subplot(1, 3, 2)
plt.plot(fpr, tpr, label = f'AUC = {roc_auc:.2f}', color = 'darkorange')
plt.title('ROC Curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
plt.grid(True)

plt.subplot(1, 3, 3)
plt.plot(recall, precision, color='green')
plt.title('Precision-Recall Curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.grid(True)

plt.tight_layout()
plt.show()

**Observation**   
Well! The class imbalance is clearly heavily affecting the ***classification*** performance of the model as made clear by the discrepancy between the recall and precision of the model between both classes.

However, the goal isn't classification, it's ***lead scoring*** and the model has a high AUC meaning that it consistently ranks those who converted higher than those who didn't. It also has a relatively high recall at the cost of more false-positives, which is talk for classification.

After the subsequent steps mentioned below (especially alignment), I will checkout the gains and lift to see if they're reasonable and if I'm satisfied with them. If not, I will explore data augmentation (over & undersampling). I already explored model selection in cells now deleted, and the best performances were relatively similar to that of the rf model used.


**Subsequently**
- I will create a SHAP plot to see what this naive model says about decision influencing factors
- I will align predicted propensity scores with ideal values and plot reliability graph before/after alignment 
- I will assign propensity scores and continue with gains and lift related work

If you want to better understand what stories preciaion and recall tell, you are welcome to checkout [this post](https://www.linkedin.com/posts/einstein-ebereonwu_%F0%9D%90%8F%F0%9D%90%AB%F0%9D%90%9E%F0%9D%90%9C%F0%9D%90%A2%F0%9D%90%AC%F0%9D%90%A2%F0%9D%90%A8%F0%9D%90%A7-%F0%9D%90%A8%F0%9D%90%AB-%F0%9D%90%91%F0%9D%90%9E%F0%9D%90%9C%F0%9D%90%9A%F0%9D%90%A5%F0%9D%90%A5-%F0%9D%90%96%F0%9D%90%A1%F0%9D%90%9A%F0%9D%90%AD-activity-7323300180882079744-4q8y?utm_source=share&utm_medium=member_desktop&rcm=ACoAACfSx2YBBgHwK5o8dtbmATIL2EtooLGRKYY). The model 

In [ ]:
base_rf_explainer = shap.Explainer(rf_model)
shap_values = base_rf_explainer(X_test)

In [ ]:
shap_values_class1 = shap_values[..., 1] #<-- actually just learnt about the use of elipse.
shap.plots.beeswarm(shap_values_class1)

In [ ]:
# Mapping values for context
housing_mapping = {cls: i for i, cls in enumerate(label_encoders['housing'].classes_)}
contact_mapping = {cls: i for i, cls in enumerate(label_encoders['contact'].classes_)}
print(f'Housing loan?: {housing_mapping}')
print(f'Contact mtd. mapping: {contact_mapping}')

**Observation**  

The SHAP plot above provides a more detailed explanation of feature contributions compared to the correlation plots from few cells above. It shows the most releant features according to the model, as well as the effect of low/high values of them.

- ***Duration***: is the most influential feature. Longer call durations significantly increase the likelihood of a client subscribing. Almost all high-duration interactions contribute positively to the model output, indicating a strong association between longer calls and successful subscriptions.

- ***Housing***: clients without a housing loan are more likely to subscribe. The model assigns positive SHAP values to such individuals, suggesting that not having a housing loan correlates with a greater chance of conversion.

- ***Month***: the month of contact shows varied influence. Some months (likely those with higher values i.e May - December) are positively associated with subscription, while others are not. Could be a seasonal trend thing...

- ***Contact***: the method of contact (cellular, telephone, or unknown) impacts subscription likelihood. Those contacted by cellular are more likely to convert. 

- ***Pdays***: lower values (recent contact) tend to decrease the likelihood of subscription, while higher values (less recent contact or no contact) are associated with positive SHAP values, suggesting that too frequent prior contact may reduce effectiveness... Let's not forget the recall of the model. This may be a misunderstanding and why the model is performing badly. Who knows..

- ***Age***: older clients tend to have slightly more positive SHAP values. This makes sense, as middle-aged or older clients are more likely to subscribe, seeing it as a solid long-term investment.

- ***Previous***: a higher number of contacts in previous campaigns slightly increases the likelihood of conversion... Maybe they finally warmed up to the product.

- ***Poutcome***: a successful outcome of a previous campaign contributes positively to the prediction. Clients with past positive responses are more likely to subscribe again... Loyal people <3

- ***Day***: early days of month seem to have a slightly higher probability of conversion while some mid-late days also have some level of positive effect. Maybe those are days within special months correlating with the seasons?a

- ***Other Features***: the other features still contribute cumulatively to the model’s decision-making process... Will experiment with and without those features to measure performance.   


**Reccomendations** 
The information above could be used for future A/B testing, reaching out to older/younger, prople who converted in the past/those who didn't, those with/without housing loan. e.t.c.

## Checking effect of model calibration

In [ ]:
calibrated_model = CalibratedClassifierCV(rf_model, cv = 'prefit', method = 'sigmoid')
calibrated_model.fit(X_calib, y_calib)

y_proba_uncalibrated = rf_model.predict_proba(X_test)[:, 1]
y_proba_calibrated = calibrated_model.predict_proba(X_test)[:, 1]

plt.figure(figsize = (8, 6))
prob_true_before, prob_pred_before = calibration_curve(y_test, y_proba_uncalibrated, n_bins = 10)
prob_true_after, prob_pred_after = calibration_curve(y_test, y_proba_calibrated, n_bins = 10)

plt.plot(prob_pred_before, prob_true_before, marker = 'x', label = 'Before Calibration', color = 'blue')
plt.plot(prob_pred_after, prob_true_after, marker = 'o', label = 'After Calibration', color = 'green')
plt.plot([0, 1], [0, 1], linestyle = '--', label = 'Ideal', color = 'red')

plt.title('Reliability Diagram (Before and After Calibration)')
plt.xlabel('Predicted Probability')
plt.ylabel('Empirical Probability')
plt.legend()
plt.grid(True)

brier_before = brier_score_loss(y_test, y_proba_uncalibrated)
brier_after = brier_score_loss(y_test, y_proba_calibrated)
print(f'Brier score before calibration: {brier_before:.4f}')
print(f'Brier score after calibration: {brier_after:.4f}')
print(f'Improvement: {(1 - brier_after/brier_before) * 100:.2f}%')

plt.show()

**Observation**  

Calibration has great effect on the predicted probabilities, thus, I will go ahead to assign propensity scores with the calibrated model as that better reflects reality.

# Training final model and continuing with lead scoring

In [ ]:
# Final model on all data, excluding calibration set
# since calibration had quite the impact when I was trying to
# understand model performance

X_combined = pd.concat([X_train, X_test], axis = 0)
y_combined = pd.concat([y_train, y_test], axis = 0)
all_data_excluding_calib = pd.concat([X_combined, y_combined], axis = 1)

rf_prop_model = RandomForestClassifier(
    n_estimators = 300,
    max_depth = 10,
    class_weight = 'balanced',
    random_state = 42)
rf_prop_model.fit(X_combined, y_combined)

calibrated_model = CalibratedClassifierCV(rf_prop_model, cv = 'prefit', method = 'sigmoid')
calibrated_model.fit(X_calib, y_calib)


# Assigning calibrated lead scores
all_data_excluding_calib['lead_score'] = calibrated_model.predict_proba(all_data_excluding_calib.drop('y', axis = 1))[:, 1]
scored_df = all_data_excluding_calib.sort_values('lead_score', ascending = False)
scored_df.head(2)

In [ ]:
total_positives = y.sum()

percentiles = np.linspace(0.01, 1, 100)
random_gain = (total_positives / len(scored_df)) * 100
gains = []
lifts = []


for top_percent in percentiles:
    top_n = int(len(scored_df) * top_percent)
    top_leads = scored_df.head(top_n)
    
    # positives captured in the top leads, gains and lift
    positives_in_top = top_leads['y'].sum()
    gain = (positives_in_top / total_positives) * 100
    lift = gain / random_gain
    
    gains.append(gain)
    lifts.append(lift)


plt.figure(figsize = (14, 5))
plt.subplot(1, 2, 1)
plt.plot(percentiles * 100, gains, color = 'blue', label = 'Normalized Gains')
plt.plot([0, 100], [random_gain, random_gain], linestyle = '--', color = 'gray', label = 'Random')
plt.title('Normalized Gains Curve')
plt.xlabel('Top Percentage of Leads')
plt.ylabel('Gains (%)')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(percentiles * 100, lifts, color = 'green', label = 'Lift Curve')
plt.title('Lift Curve')
plt.xlabel('Top Percentage of Leads')
plt.ylabel('Lift')
plt.legend()

plt.tight_layout()
plt.show()

**Observation**  

The Gains curve clearly indicates that the majority of conversions are concentrated within the top 20% to 40% of leads. Anything after this range, results is almost no growth of gains, suggesting diminishing returns from targeting lower-ranked leads.   

The lift curve also shows similar information, with diminishing returns as population increases.

In [ ]:
total_positives = y.sum()

top_percent = 0.40
top_n = int(len(scored_df) * top_percent)

top_leads = scored_df.head(top_n)
positives_in_top = top_leads['y'].sum()

# Calculate gains and lift
random_gain = (total_positives / len(scored_df)) * 100
gains = (positives_in_top / total_positives) * 100
lift = gains / random_gain


print('#' * 81)
print('\t\t\t\t**FINAL NOTES**')
print(f'{gains:.2f}% of all conversions are captured by just the top {top_percent * 100}% of leads,')
print(f'and they are {lift:.2f} times more likely to convert than if we picked leads at random.')
print('#' * 81)

# Final Notes: Term Deposit Propensity Modeling

This analysis focused on optimising telephonic marketing campaigns for term deposit products by leveraging a robust propensity modeling approach. The goal was to identify high-likelihood conversion targets, thereby increasing campaign efficiency and ROI for the bank which led to the discovery of the fact that `83.54%` of all conversions are captured by just the top ***40.0%*** of leads, and they are `6.01` times more likely to convert than if leads are picked at random.

**Key Findings**

- **Past Campaign Effectiveness**: The historical conversion rate was 11.7%, aligning with industry norms for random targeting in direct marketing.
- **Conversion Drivers**:
    - *Demographics*: Older clients (30–60 years), those in management roles, singles, and those with tertiary education were more likely to subscribe.
    - *Financial Status*: Higher account balances correlated with higher subscription rates.
    - *Behavioral Factors*:
        - Longer call durations were strongly associated with successful conversions.
        - Fewer campaign contacts (1–3) were optimal; conversion rates dropped with excessive contact attempts.
        - Recent prior contact (low `pdays`) and positive outcomes from previous campaigns (`poutcome`) significantly boosted conversion likelihood.
    - *Channel and Timing*: Cellular contact outperformed telephone, and May was the most successful month for subscriptions.
    - *Credit Status*: Clients without existing loans or defaults were more receptive.

**Modeling \& Data Handling**

- Outliers in call duration, campaign contacts, and previous contacts were carefully excluded to prevent skewed modeling.
- Categorical variables were encoded with attention to ordinal relationships (e.g., education, month).
- Data balancing techniques (SMOTE, undersampling) were considered to address class imbalance.
- Initial model was trained with a `train/test/calib` set to understand performance as well as impact of calibration on score alignment.
- Final model was trained on the previously mentioned `train + test` set, then calibrated using original calibrated set and sigmoid method to better reflect true conversion likelihoods.


## Recommendations

**1. Targeted Outreach**

- Prioritise segments with high predicted propensity: older, single, highly educated, management-level clients, and those with higher balances.
- Focus on clients with no credit defaults or active loans.

**2. Optimise Campaign Strategy**

- Limit contact attempts to a maximum of 3 per client to avoid diminishing returns and potential customer fatigue.
- Schedule campaigns to peak in May or other high-performing months identified in the analysis.
- Favor cellular over telephone outreach for higher engagement.

**3. Personalise Messaging**

- Tailor scripts and offers for high-propensity segments (e.g., highlight product benefits relevant to management professionals or retirees).
- For clients with prior positive campaign outcomes, reference past interactions to build rapport.

**4. Enhance Data Quality \& Monitoring**

- Regularly review and update customer data to maintain segmentation accuracy.
- Try to find out the education levels an outreach method of those currently unknown
- Monitor for outliers and update exclusion thresholds as needed to keep the model robust.


## A/B Testing Recommendations

To validate and further optimise these strategies, implement the following A/B tests:


| Test Focus | Group A (Control) | Group B (Test) | Success Metric |
| :-- | :-- | :-- | :-- |
| Contact Channel | Telephone outreach | Cellular outreach | Conversion rate |
| Campaign Timing | Standard months | May (peak month) | Conversion rate |
| Contact Frequency | Up to 6 contacts | Max 3 contacts | Conversion rate, opt-out rate |
| Personalisation | Generic script | Tailored script for high-propensity segments | Conversion rate, call duration |
| Prior Campaign Targeting | All clients | Only clients with prior positive outcomes | Conversion rate |

- **Monitor**: Conversion rates, call durations, and opt-out rates for each test group.
- **Iterate**: Refine targeting and messaging based on statistical significance and observed lift.


## Final Observations

- The propensity model provides actionable insights for resource allocation and campaign design, enabling the bank to focus efforts on clients most likely to convert.
- Regular calibration and monitoring are essential to adapt to changing customer behaviors and market conditions.
- Combining data-driven targeting with thoughtful A/B testing will maximise marketing ROI and customer satisfaction.

**NOTE**: This entire analysis is part of my weekly series in efforts to **demystify applied statistical techniques through real-world, project-driven examples**, making concepts like propensity modelling, causal inference, and evaluation metrics more accessible to practitioners of all backgrounds.

# Let's connect!
- [X](https://x.com/einsteinmuna)
- [GitHub](https://github.com/munas-git)
- [LinkedIn](https://www.linkedin.com/in/einstein-ebereonwu/)